In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from keras.utils import np_utils

%matplotlib inline

import check_miss_value


In [3]:
# picle ファイルから読み込み
train = pd.read_pickle('./pd_train.pk2')
test = pd.read_pickle('./pd_test.pk2')

In [4]:
# train Survived あり
# test PassengerId あり
train.shape, test.shape

((891, 8), (418, 8))

In [5]:
train.corr()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Survived,1.000000,-0.338481,-0.543351,-0.064910,-0.035322,0.081629,0.257307,-0.167675
Pclass,-0.338481,1.000000,0.131900,-0.339898,0.083081,0.018443,-0.549500,0.162098
Sex,-0.543351,0.131900,1.000000,0.081163,-0.114631,-0.245489,-0.182333,0.108262
Age,-0.064910,-0.339898,0.081163,1.000000,-0.233296,-0.172482,0.096688,-0.018754
SibSp,-0.035322,0.083081,-0.114631,-0.233296,1.000000,0.414838,0.159651,0.068230
Parch,0.081629,0.018443,-0.245489,-0.172482,0.414838,1.000000,0.216225,0.039798
Fare,0.257307,-0.549500,-0.182333,0.096688,0.159651,0.216225,1.000000,-0.224719
Embarked,-0.167675,0.162098,0.108262,-0.018754,0.068230,0.039798,-0.224719,1.000000


In [6]:
train.corr()[train.corr() > 0.2]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Survived,1.000000,NaN,NaN,NaN,NaN,NaN,0.257307,NaN
Pclass,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
Sex,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
Age,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
SibSp,NaN,NaN,NaN,NaN,1.000000,0.414838,NaN,NaN
Parch,NaN,NaN,NaN,NaN,0.414838,1.000000,0.216225,NaN
Fare,0.257307,NaN,NaN,NaN,NaN,0.216225,1.000000,NaN
Embarked,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [7]:
train.corr()[train.corr() < -0.2]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Survived,NaN,-0.338481,-0.543351,NaN,NaN,NaN,NaN,NaN
Pclass,-0.338481,NaN,NaN,-0.339898,NaN,NaN,-0.549500,NaN
Sex,-0.543351,NaN,NaN,NaN,NaN,-0.245489,NaN,NaN
Age,NaN,-0.339898,NaN,NaN,-0.233296,NaN,NaN,NaN
SibSp,NaN,NaN,NaN,-0.233296,NaN,NaN,NaN,NaN
Parch,NaN,NaN,-0.245489,NaN,NaN,NaN,NaN,NaN
Fare,NaN,-0.549500,NaN,NaN,NaN,NaN,NaN,-0.224719
Embarked,NaN,NaN,NaN,NaN,NaN,NaN,-0.224719,NaN
